## Imports

In [ ]:
import ultralytics, cv2, os, yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
ultralytics.checks()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Create YAML file

In [ ]:
cell_yaml = dict(
    train =r'C:\Users\antoi\Documents\GitHub\Cell_Detector\Test_Yolo\datasets\images\train',
    val =r'C:\Users\antoi\Documents\GitHub\Cell_Detector\Test_Yolo\datasets\images\val',
    test =r'C:\Users\antoi\Documents\GitHub\Cell_Detector\Test_Yolo\datasets\images\test',
    nc =1,
    names =['cell']
)

with open('cell.yaml', 'w') as outfile:
    yaml.dump(cell_yaml, outfile, default_flow_style=True)

## Train

In [ ]:
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

In [ ]:
# with open('cell.yaml', 'r') as file:
#     yaml.load(cell_yaml, file.read())

model.train(data = 'cell.yaml', epochs=10, imgsz=640, patience = 50, dropout=0.2, project = 'train_local', device='cpu')

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/cell.yaml epochs=10 show=True max_det=600 imgsz=640 dropout=0.2 project=/content/drive/MyDrive/Colab_Notebooks/datasets/results name=train

## Test

In [ ]:
best_path='train_local/train7/weights/best.pt'
source='datasets/images/test'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths)

In [ ]:
model_trained = YOLO(best_path)

In [ ]:
!yolo task=detect mode=predict model={best_path} conf=0.004 source={source}

In [ ]:
results = model_trained(source, conf=0.01)
plot = results[0].plot(line_width = 1, font_size = 2)

# cv2.imshow('result', plot)

print(len(results[0]))

cv2.imwrite('result.jpg', plot)